<a href="https://colab.research.google.com/github/Aftabgazali/Linear-Regression-Using-Pytorch/blob/main/Linear_Regression_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Setting up the Device Agnostic Code

***Note:*** *If using CUDA then below are the default changes you gotta make before the training loop.*

`model.to(device)`
`X_train = X_train.to(device)`
`y_train = y_train.to(device)`
`X_test = X_test.to(device)`
`y_test = y_test.to(device)`

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Colab is running in {device}")

# Importing the Datasets

In [ ]:
from sklearn.datasets import fetch_california_housing
house_price_data = fetch_california_housing()

In [ ]:
df = pd.DataFrame(house_price_data.data)
df.columns = house_price_data.feature_names
print(df)

In [ ]:
df['Target'] = house_price_data.target
df.head()

In [ ]:
# Do it in one go
# from sklearn import datasets
# diabetes_X, diabetes_y = datasets.fetch_california_housing(return_X_y=True)

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
type(y)

# Prepare the Data into Tensors

In [ ]:
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).float()

In [ ]:
y_tensor.dtype

# Splitting the Data into Training & Testing Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tensor,y_tensor, test_size=0.33, random_state=42)
y_train.dtype

# Model Building

***Note:*** *`in_features` is the number of in-dependent features in your datasets & `out_features` in the number of dependent features in your datasets*

In [ ]:
class LinearRegressionModel_V2(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_layer = nn.Linear(in_features=8, # Creating the Linear Layer applying Linear Transform(Linear Regression: y = wT*x + b)
                                  out_features=1) # This is equivalent to creating the weight & bias and assigning them with a random values at start

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x) # we then pass the training samples to the linear layer to update the weight & the bias by learning from the training samples.

# Set the Manual Seed
torch.manual_seed(42)
model = LinearRegressionModel_V2()
model.state_dict()

# Training the Model

In [ ]:
# Assign the Loss function & the Optimizers
model_loss = nn.L1Loss() # Mean Squared Error / Cost Function

model_optimizer = torch.optim.Adam(params=model.parameters(),lr=0.01)


In [ ]:
# Training Loop
# For setting up the Model as well as our Samples to CDU Device
model.to(device)
epochs = 200
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
for epoch in range(epochs):
  # Training Mode:
  model.train()

  # Train the Model
  train_predictions = model(X_train)

  # Training Loss
  training_loss = model_loss(train_predictions, y_train)

  # Set the grad to zero
  model_optimizer.zero_grad()

  # Backpropagation:
  training_loss.backward()

  # Progress/step the optimizer (to go to the global minima) # Update the model parameters value to better represent the b.f.
  model_optimizer.step()

  # Testing Mode:
  model.eval()
  with torch.inference_mode():
    testing_predictions = model(X_test)
    testing_loss = model_loss(testing_predictions,y_test)
    if epoch % 10 == 0:
      print(f"Epoch {epoch} | Training Loss {training_loss} | Testing Loss {testing_loss}")


# Testing the Model Predictions

In [ ]:
model.eval()
with torch.inference_mode():
  y_predictions = model(X_test)

len(y_predictions)

In [ ]:
epochs = np.arange(1,6813,1)
len(epochs)

In [ ]:
def plot_predictions(train_labels=y_test,predicted_labels=y_predictions,epochs=epochs):
  plt.figure(figsize=(15,12))
  plt.plot(epochs[::50],train_labels[::50].cpu(),c="b",label="Training Test Set")
  plt.plot(epochs[::50],predicted_labels[::50].cpu(),c="g",label="Predicted Test Set")
  plt.legend()
  plt.show()

plot_predictions(y_test,y_predictions,epochs)

# Save the Model

In [ ]:
import os

MODEL_PATH = "model"
if not os.path.exists(MODEL_PATH):
    os.mkdir(MODEL_PATH)

MODEL_NAME = "linear_regression_model_v2.pth"
MODEL_SAVE_PATH = os.path.join(MODEL_PATH, MODEL_NAME)

print(f"Saving the V2 version of the Model to {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(),f=MODEL_SAVE_PATH)